<a href="https://colab.research.google.com/github/krikorantranik/Work/blob/main/AsscRules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I want to show an example of the use of market basket analysis techniques (the apriori algorithm), but I will use a different example than the classical suggestion to add items to a shopping card. In this case, I will use the list of air traffic routes by airlines to suggest, for a given set of two airports, which route makes more "sense" to be added, given what existing airlines already do.

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns

Import the raw data and inspect it.

In [2]:
origset = pd.read_csv("routes.csv")
origset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Airline,Airline ID,Source Airport,Source Airport ID,Destination Airport,Destination Airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


I format that data so that it has the structure of a list of items and transactions. The transactions are between the airlines and the airports.

In [3]:
origsetA = origset[['Airline','Source Airport']]
origsetA.columns = ['Airline','Airport']
origsetB = origset[['Airline','Destination Airport']]
origsetB.columns = ['Airline','Airport']
newset = pd.concat([origsetA.reset_index(drop=True),origsetB.reset_index(drop=True)], ignore_index=True).drop_duplicates().dropna()
newset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Airline,Airport
0,2B,AER
1,2B,ASF
3,2B,CEK
5,2B,DME
9,2B,EGO
...,...,...
128688,VA,ZQN
128875,VA,HKG
129363,VS,SJC
131288,WN,PWM


The input to the algorithm must be presented in form of a table of 0 and 1's.

In [4]:
newset = newset.pivot_table(index='Airline', columns='Airport', aggfunc=len, fill_value=0)
newset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Airport,AAE,AAL,AAN,AAQ,AAR,AAT,AAX,AAY,ABA,ABB,...,ZSA,ZSE,ZSJ,ZTB,ZTH,ZUH,ZUM,ZVK,ZYI,ZYL
Airline,,,,,,,,,,,,,,,,,,,,,
2B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2G,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2J,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2K,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
ZI,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


This is the core of the algorithm. I produce the rules based on a minimum value of support and format lists with the airport codes.

In [5]:
rules_m = apriori(newset, min_support = 0.045, use_colnames = True)
rules = association_rules(rules_m, metric ="lift")
rules = rules.sort_values(['confidence'], ascending =[False])
rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
rules['consequents'] = rules['consequents'].apply(lambda x: list(x))
rules['lenantecedents'] = rules['antecedents'].apply(lambda x: len(x))
rules['lenconsequents'] = rules['consequents'].apply(lambda x: len(x))
rules = rules[rules['lenantecedents']>1]
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,lenantecedents,lenconsequents
650796,"[PVG, HKG, KIX, NRT, LHR]",[SIN],0.047535,0.146127,0.047535,1.000000,6.843373,0.040589,inf,0.896488,5,1
533151,"[HKG, NRT, CGK]","[BKK, SIN, PVG]",0.045775,0.070423,0.045775,1.000000,14.200000,0.042551,inf,0.974170,3,3
533158,"[SIN, NRT, CGK]","[HKG, BKK, PVG]",0.045775,0.079225,0.045775,1.000000,12.622222,0.042148,inf,0.964945,3,3
533170,"[NRT, CGK]","[HKG, BKK, SIN, PVG]",0.045775,0.068662,0.045775,1.000000,14.564103,0.042632,inf,0.976015,2,4
533182,"[PEK, PVG, HKG, CGK, BKK]",[SIN],0.045775,0.146127,0.045775,1.000000,6.843373,0.039086,inf,0.894834,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
439174,"[FRA, CDG]","[BCN, PEK, LHR, BKK]",0.130282,0.047535,0.045775,0.351351,7.391391,0.039582,1.468383,0.994239,2,4
171237,"[FRA, CDG]","[MAD, VIE, ARN]",0.130282,0.054577,0.045775,0.351351,6.437663,0.038664,1.457526,0.971193,2,3
589334,"[FRA, CDG]","[DUS, MXP, ZRH, MUC]",0.130282,0.052817,0.045775,0.351351,6.652252,0.038894,1.460241,0.976954,2,4
299727,"[FRA, CDG]","[IST, VIE, ZRH]",0.130282,0.051056,0.045775,0.351351,6.881640,0.039123,1.462955,0.982716,2,3


Pivoting the table to query on the list of input airports.

In [6]:
rules['ListAntecedents'] = rules['antecedents'].apply(lambda x: '(' + ','.join(x) + ')')
rules['ListConsequents'] = rules['consequents'].apply(lambda x: '(' + ','.join(x) + ')')
search = rules[['antecedents']]
search = search.explode(['antecedents'])
search = search.pivot_table(index=search.index, columns='antecedents', aggfunc=len, fill_value=0)
search['Total'] = search.sum(axis = 1)
search

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


antecedents,AGP,AMS,ARN,ATH,ATL,AUH,BAH,BCN,BKK,BLQ,...,TPE,TXL,VCE,VIE,WAW,XIY,XMN,YYZ,ZRH,Total
2218,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2219,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2220,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2224,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2225,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942380,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
942381,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
942382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
942383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


I define this function for convenience. Then inspect a couple of airport combinations.

In [10]:
def get_out(airport1, airport2, onlyExact, minSupport, minConfidence, minLift, numRec):
 idx = search[search[airport1]==1]
 idx = idx[idx[airport2]==1]
 if onlyExact == True:
  idx = idx[idx['Total']==2]
 out = rules.loc[idx.index]
 out = out[out['confidence']>=minConfidence]
 out = out[out['support']>=minSupport]
 out = out[out['lift']>=minLift]
 out = out[out['lenconsequents']<=numRec]
 out = out.sort_values('confidence', ascending=False)
 return out[['antecedents','consequents','support','confidence','lift']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


So, I can use the function to input two airports, report only the "baskets" that have only those two airports, with minimum support, confidence, and lift, and only report one airport back.

In [20]:
result = get_out(airport1 = 'AMS', airport2 = 'FRA', onlyExact = True, minSupport = 0.05, minConfidence = 0.1, minLift = 1, numRec=1)
result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,support,confidence,lift
3336,"[AMS, FRA]",[FCO],0.086268,0.875000,5.402174
2986,"[AMS, FRA]",[CDG],0.084507,0.857143,4.466579
3514,"[AMS, FRA]",[LHR],0.082746,0.839286,5.543189
3598,"[AMS, FRA]",[ZRH],0.082746,0.839286,6.621032
2550,"[AMS, FRA]",[BCN],0.077465,0.785714,6.375510
3532,"[AMS, FRA]",[MUC],0.075704,0.767857,5.664193
2782,"[AMS, FRA]",[BRU],0.073944,0.750000,6.761905
3580,"[AMS, FRA]",[VIE],0.070423,0.714286,5.795918
3538,"[AMS, FRA]",[MXP],0.070423,0.714286,5.409524
3526,"[AMS, FRA]",[MAD],0.066901,0.678571,5.585921


According to this, for an airline that has flights to Amsterdam and Frankfurt, the most sense is to add Rome, Paris, London, and/or Zurich.